API key: RnHS3Y3lkrv9pzkHVFcVYVArH \\
API secret key: xLMxTxhEHluPlcsPgb2rur07hBR4qSuzGdIVhJg0hpEiR1nTBm \\
Bearer token: 
AAAAAAAAAAAAAAAAAAAAAJ6bOAEAAAAAx7r6xai0HEcEhCREYGRBCePMops%3DnI0dVPhA0eFSBFrlc6M33YznpLELExxZGq0Mv3ZTXUTaj8C7Sq

##Q1